In [2]:
#!pip install web3

In [19]:
import pandas as pd
import requests
import datetime
import time
import json
from web3 import Web3 as web3
import numpy as np
from web3 import Web3

In [ ]:
# Initialize a Web3 instance
infura_project_id = 'YOUR-PROJECT-ID'
infura_url = f'https://mainnet.infura.io/v3/{infura_project_id}'

web3 = Web3(Web3.HTTPProvider(infura_url))

In [102]:
#introduce a caching layer to reduce API calls

from web3.middleware import geth_poa_middleware, simple_cache_middleware
# add the geth_poa_middleware to handle POA chains
web3.middleware_onion.inject(geth_poa_middleware, layer=0)

# add the simple_cache_middleware to cache responses
web3.middleware_onion.add(simple_cache_middleware)

In [9]:
#load top token addresses and block numbers that will be needed later on
top_tokens_addresses = pd.read_csv('../../utils/price-data/top_tokens_addresses.csv') 
block_numbers_df = pd.read_csv('../../utils/block-numbers/block_numbers_2020-01-2023-04.csv')

### retrieve creation TX hashes for the escrow contracts

so we can obtain the tx hash and timestamp from the day they were deployed

In [7]:
import requests

# Replace with your etherscan API key
api_key = "YOUR-API-KEY"

# List of contract addresses to query
contract_addresses = [
    "0xcEe284F754E854890e311e3280b767F80797180d", #deployed Jun-16-2021
    "0xa3A7B6F88361F48403514059F1F16C8E78d60EeC", #deployed Jun-15-2021
    "0x8315177aB297bA92A06054cE80a67Ed4DBd7ed3a", #deployed Aug-25-2022 
    "0xA10c7CE4b876998858b1a9E12b10092229539400"#deployed Sep-20-2021
]

# API endpoint
url = "https://api.etherscan.io/api"

# Query parameters
params = {
    "module": "contract",
    "action": "getcontractcreation",
    "contractaddresses": ",".join(contract_addresses),
    "apikey": api_key
}

# Send GET request
response = requests.get(url, params=params)

# Extract transaction hash for each contract address
result_list = response.json()["result"]

creation_txHash_0xcEe284F754E854890e311e3280b767F80797180d = result_list[0]["txHash"]
creation_txHash_0xa3A7B6F88361F48403514059F1F16C8E78d60EeC = result_list[1]["txHash"]
creation_txHash_0x8315177aB297bA92A06054cE80a67Ed4DBd7ed3a = result_list[2]["txHash"]
creation_txHash_0xA10c7CE4b876998858b1a9E12b10092229539400 = result_list[3]["txHash"]

# Print transaction hash for each contract address
print("Transaction hash for Contract 1:", creation_txHash_0xcEe284F754E854890e311e3280b767F80797180d)
print("Transaction hash for Contract 2:", creation_txHash_0xa3A7B6F88361F48403514059F1F16C8E78d60EeC)
print("Transaction hash for Contract 3:", creation_txHash_0x8315177aB297bA92A06054cE80a67Ed4DBd7ed3a)
print("Transaction hash for Contract 4:", creation_txHash_0xA10c7CE4b876998858b1a9E12b10092229539400)


Transaction hash for 0xcEe284F754E854890e311e3280b767F80797180d: 0x334a8c4172e3adfa0911c82de7b13ad9f18c75059ad8091f4d2659293510298e
Transaction hash for 0xa3A7B6F88361F48403514059F1F16C8E78d60EeC: 0xaeeb4daf285b38b858d585e029405ee777a4b7882bf6a66cbfb3a53f6dbf7253
Transaction hash for 0x8315177aB297bA92A06054cE80a67Ed4DBd7ed3a: 0xd52bb370667ae76f14c7d65b4e262cf72c86ec51bfc1186175f806279a4bd7d6
Transaction hash for 0xA10c7CE4b876998858b1a9E12b10092229539400: 0x248787fbf0415ff1da68ba618da0cbdd5dcc7c6029be38709f95ea837211edc5


<h1 style="color: blue; font-size: 36px; font-family: Arial;">L1ERC20Gateway Escrow</h1>


### retrieve block number and timestamp for the deployment of the contract

this way i can start querying the token balance from that day on

In [50]:
api_key = 'YOUR-API-KEY' # replace with your Etherscan API key
tx_hash = creation_txHash_0xa3A7B6F88361F48403514059F1F16C8E78d60EeC # replace with the transaction hash you want to retrieve

# make the API request
url = f'https://api.etherscan.io/api?module=proxy&action=eth_getTransactionByHash&txhash={tx_hash}&apikey={api_key}'
response = requests.get(url)

# parse the response to retrieve the block number
result = response.json()['result']
block_number = int(result['blockNumber'], 16)

print(f'The deployment transaction {tx_hash} for the smart contract  was mined in block number {block_number}.')

# make the API request
url = f'https://api.etherscan.io/api?module=block&action=getblockreward&blockno={block_number}&apikey={api_key}'
response = requests.get(url)

# parse the response to retrieve the timestamp
response_json = response.json()
if response_json['status'] == '1':
    result = response_json['result']
    timestamp = int(result['timeStamp'])
    datetime_obj = datetime.datetime.fromtimestamp(timestamp)
    human_readable_time = datetime_obj.strftime('%Y-%m-%d %H:%M:%S')
    print(f'The block with number {block_number} was mined at timestamp {human_readable_time}.')
else:
    print(f'There was an error: {response_json["result"]}')




The deployment transaction 0xaeeb4daf285b38b858d585e029405ee777a4b7882bf6a66cbfb3a53f6dbf7253 for the smart contract 0xa3A7B6F88361F48403514059F1F16C8E78d60EeC was mined in block number 12640867.
The block with number 12640867 was mined at timestamp 2021-06-15 19:08:20.


### retreive all tokens held by the L1ERC20Gateway escrow holding ERC20 Tokens


In [1]:
#contract-address: 0xa3A7B6F88361F48403514059F1F16C8E78d60EeC
#created at Jun-15-2021 #at block 12640867


In [4]:
import pandas as pd
import requests

# Replace API_KEY with your own Ethplorer API key
API_KEY = "YOUR-API-KEY"

# Replace CONTRACT_ADDRESS with the address of the contract you want to query
CONTRACT_ADDRESS = "0xa3A7B6F88361F48403514059F1F16C8E78d60EeC"

# Construct the API endpoint URL
url = f"https://api.ethplorer.io/getAddressInfo/{CONTRACT_ADDRESS}?apiKey={API_KEY}"

# Send the API request and parse the response as JSON
response = requests.get(url)
data = response.json()

# Get the list of token balances held by the contract
token_balances = data.get("tokens", [])

# Create an empty DataFrame
token_addresses_held_L1ERC20Gateway = pd.DataFrame(columns=["Token Address"])

# Append each token address to a new row in the DataFrame
for token_balance in token_balances:
    token_address = token_balance.get("tokenInfo", {}).get("address")
    token_addresses_held_L1ERC20Gateway = token_addresses_held_L1ERC20Gateway.append({"Token Address": token_address}, ignore_index=True)

# Print the DataFrame
print(token_addresses_held_L1ERC20Gateway)


                                  Token Address
0    0x0ce51000d5244f1eaac0b313a792d5a5f96931bf
1    0x6810e776880c02933d47db1b9fc05908e5386b96
2    0x9f8f72aa9304c8b593d555f12ef6589cc3a579a2
3    0x514910771af9ca656af840dff83e8264ecf986ca
4    0x1f9840a85d5af5bf1d1762f925bdaddc4201f984
..                                          ...
602  0x745407c86df8db893011912d3ab28e68b62e49b0
603  0x85f6eb2bd5a062f5f8560be93fb7147e16c81472
604  0x18084fba666a33d37592fa2633fd49a74dd93a88
605  0x4da27a545c0c5b758a6ba100e3a049001de870f5
606  0xcfeaead4947f0705a14ec42ac3d44129e1ef3ed5

[607 rows x 1 columns]


all token_addresses held by the escrow that also appear in the top 300 of CoinGecko

In [10]:
# all token_addresses held by the escrow that also appear in the top 300 of CoinGecko
L1ERC20Gateway_top_token_addresses = pd.merge(top_tokens_addresses, token_addresses_held_L1ERC20Gateway, left_on="token_address", right_on="Token Address")
L1ERC20Gateway_top_token_addresses['token_address'] = L1ERC20Gateway_top_token_addresses['token_address'].apply(lambda x: web3.to_checksum_address(x))


In [11]:
L1ERC20Gateway_top_token_addresses

,coingecko_id,token_address,decimals,Token Address
0,binancecoin,0xB8c77482e45F1F44dE1745F52C74426C631bDD52,18,0xb8c77482e45f1f44de1745f52c74426c631bdd52
1,matic-network,0x7D1AfA7B718fb893dB30A3aBc0Cfc608AaCfeBB0,18,0x7d1afa7b718fb893db30a3abc0cfc608aacfebb0
2,shiba-inu,0x95aD61b0a150d79219dCF64E1E6Cc01f0B64C4cE,18,0x95ad61b0a150d79219dcf64e1e6cc01f0b64c4ce
3,binance-usd,0x4Fabb145d64652a948d72533023f6E7A623C7C53,18,0x4fabb145d64652a948d72533023f6e7a623c7c53
4,uniswap,0x1f9840a85d5aF5bf1D1762F925BDADdC4201F984,18,0x1f9840a85d5af5bf1d1762f925bdaddc4201f984
...,...,...,...,...
88,alethea-artificial-liquid-intelligence-token,0x6B0b3a982b4634aC68dD83a4DBF02311cE324181,18,0x6b0b3a982b4634ac68dd83a4dbf02311ce324181
89,coti,0xDDB3422497E61e13543BeA06989C0789117555c5,18,0xddb3422497e61e13543bea06989c0789117555c5
90,ankreth,0xE95A203B1a91a908F9B9CE46459d101078c2c3cb,18,0xe95a203b1a91a908f9b9ce46459d101078c2c3cb
91,republic-protocol,0x408e41876cCCDC0F92210600ef50372656052a38,18,0x408e41876cccdc0f92210600ef50372656052a38


## retreive ERC20 token balance for the L1ERC20Gateway escrow 

This is done in multiple batches due to long execution times

### First batch

In [ ]:

def get_token_balance(token_holding_address_checksum, token_address, contract, block_number):
    try:
        token_balance = contract.functions.balanceOf(token_holding_address_checksum).call(block_identifier=block_number)
        has_error = False
    except Exception as e:
        token_balance = 0
        has_error = True
        #print(f"Error getting token balance: {e}")
    return token_balance, has_error

# Define the ERC20 contract ABI
abi = [
    {
        'inputs': [],
        'name': 'decimals',
        'outputs': [{'internalType': 'uint8', 'name': '', 'type': 'uint8'}],
        'stateMutability': 'view',
        'type': 'function'
    },
    {
        'inputs': [{'internalType': 'address', 'name': '', 'type': 'address'}],
        'name': 'balanceOf',
        'outputs': [{'internalType': 'uint256', 'name': '', 'type': 'uint256'}],
        'stateMutability': 'view',
        'type': 'function'
    }
]

# Define the token addresses DataFrame
token_addresses_df = L1ERC20Gateway_top_token_addresses

token_holding_address = "0xa3a7b6f88361f48403514059f1f16c8e78d60eec"
token_holding_address_checksum = web3.to_checksum_address(token_holding_address)

# Create a contract instance for each token address
contracts = {}
for _, row in token_addresses_df.iterrows():
    token_address = row['Token Address']
    token_address_checksum = web3.to_checksum_address(token_address)
    contract = web3.eth.contract(address=token_address_checksum, abi=abi)
    contracts[token_address] = contract

# Retrieve the token balances for each token address and block number, and store the results in a DataFrame
balances = []
for i, row in block_numbers_df.iloc[531:791].iterrows():
    print(f"Retrieving token balances for {row['Date']} - Block Number:{row['Block Number']} ")
    for _, row2 in token_addresses_df.iterrows(): 
        token_decimals = row2['decimals']
        token_address = row2['Token Address']
        block_number = row['Block Number']
        contract = contracts[token_address]
        token_balance, has_error = get_token_balance(token_holding_address_checksum, token_address, contract, block_number)
        #print(f"On day {row['Date']} token address {token_address} had a balance of {token_balance} ")
        #print(f"Token balance for address {token_address} at Block-Nr. {row['Block Number']} is: {token_balance}")
        time.sleep(0.5)
        balances.append({
            'date': row['Date'],
            'block_number': block_number,
            'token_address': token_address,
            'token_name': row2['coingecko_id'],
            'token_decimals': row2['decimals'],
            'balance': token_balance / 10**token_decimals,
            'has_error': has_error
        })

L1ERC20Gateway_token_balances_per_day_part01 = pd.DataFrame(balances)


In [ ]:
#L1ERC20Gateway_token_balances_per_day_part01.to_csv('L1ERC20Gateway/L1ERC20Gateway_token_balances_per_day_part01.csv', index=False)

### Second batch

In [ ]:

def get_token_balance(token_holding_address_checksum, token_address, contract, block_number):
    try:
        token_balance = contract.functions.balanceOf(token_holding_address_checksum).call(block_identifier=block_number)
        has_error = False
    except Exception as e:
        token_balance = 0
        has_error = True
        print(f"Error getting token balance for {token_address} at {block_number} because: {e}")
    return token_balance, has_error

# Define the ERC20 contract ABI
abi = [
    {
        'inputs': [],
        'name': 'decimals',
        'outputs': [{'internalType': 'uint8', 'name': '', 'type': 'uint8'}],
        'stateMutability': 'view',
        'type': 'function'
    },
    {
        'inputs': [{'internalType': 'address', 'name': '', 'type': 'address'}],
        'name': 'balanceOf',
        'outputs': [{'internalType': 'uint256', 'name': '', 'type': 'uint256'}],
        'stateMutability': 'view',
        'type': 'function'
    }
]

# Define the token addresses DataFrame
token_addresses_df = L1ERC20Gateway_top_token_addresses

token_holding_address = "0xa3a7b6f88361f48403514059f1f16c8e78d60eec"
token_holding_address_checksum = web3.to_checksum_address(token_holding_address)

# Create a contract instance for each token address
contracts = {}
for _, row in token_addresses_df.iterrows():
    token_address = row['Token Address']
    token_address_checksum = web3.to_checksum_address(token_address)
    contract = web3.eth.contract(address=token_address_checksum, abi=abi)
    contracts[token_address] = contract


# Retrieve the token balances for each token address and block number, and store the results in a DataFrame
balances = []
for i, row in block_numbers_df.iloc[790:1000].iterrows():
    print(f"Retrieving token balances for {row['Date']} - Block Number:{row['Block Number']} ")
    for _, row2 in token_addresses_df.iterrows(): 
        token_decimals = row2['decimals']
        token_address = row2['Token Address']
        block_number = row['Block Number']
        contract = contracts[token_address]
        token_balance, has_error = get_token_balance(token_holding_address_checksum, token_address, contract, block_number)
        #print(f"On day {row['Date']} token address {token_address} had a balance of {token_balance} ")
        #print(f"Token balance for address {token_address} at Block-Nr. {row['Block Number']} is: {token_balance}")
        time.sleep(0.8)
        balances.append({
            'date': row['Date'],
            'block_number': block_number,
            'token_address': token_address,
            'token_name': row2['coingecko_id'],
            'token_decimals': row2['decimals'],
            'balance': token_balance / 10**token_decimals,
            'has_error': has_error
        })

L1ERC20Gateway_token_balances_per_day_part02 = pd.DataFrame(balances)


The Error "Could not decode contract function call to balanceOf with return data" appears if a Token has not been deployed at the time of retrieving the token balance.

In [ ]:
#L1ERC20Gateway_token_balances_per_day_part02.to_csv('L1ERC20Gateway/L1ERC20Gateway_token_balances_per_day_part02.csv', index=False)

## Third Batch

In [ ]:

def get_token_balance(token_holding_address_checksum, token_address, contract, block_number):
    try:
        token_balance = contract.functions.balanceOf(token_holding_address_checksum).call(block_identifier=block_number)
        has_error = False
    except Exception as e:
        token_balance = 0
        has_error = True
        print(f"Error getting token balance for {token_address} at {block_number} because: {e}")
    return token_balance, has_error

# Define the ERC20 contract ABI
abi = [
    {
        'inputs': [],
        'name': 'decimals',
        'outputs': [{'internalType': 'uint8', 'name': '', 'type': 'uint8'}],
        'stateMutability': 'view',
        'type': 'function'
    },
    {
        'inputs': [{'internalType': 'address', 'name': '', 'type': 'address'}],
        'name': 'balanceOf',
        'outputs': [{'internalType': 'uint256', 'name': '', 'type': 'uint256'}],
        'stateMutability': 'view',
        'type': 'function'
    }
]

# Define the token addresses DataFrame
token_addresses_df = L1ERC20Gateway_top_token_addresses

token_holding_address = "0xa3a7b6f88361f48403514059f1f16c8e78d60eec"
token_holding_address_checksum = web3.to_checksum_address(token_holding_address)

# Create a contract instance for each token address
contracts = {}
for _, row in token_addresses_df.iterrows():
    token_address = row['Token Address']
    token_address_checksum = web3.to_checksum_address(token_address)
    contract = web3.eth.contract(address=token_address_checksum, abi=abi)
    contracts[token_address] = contract


# Retrieve the token balances for each token address and block number, and store the results in a DataFrame
balances = []
for i, row in block_numbers_df.iloc[1000:].iterrows():
    print(f"Retrieving token balances for {row['Date']} - Block Number:{row['Block Number']} ")
    for _, row2 in token_addresses_df.iterrows(): 
        token_decimals = row2['decimals']
        token_address = row2['Token Address']
        block_number = row['Block Number']
        contract = contracts[token_address]
        token_balance, has_error = get_token_balance(token_holding_address_checksum, token_address, contract, block_number)
        #print(f"On day {row['Date']} token address {token_address} had a balance of {token_balance} ")
        #print(f"Token balance for address {token_address} at Block-Nr. {row['Block Number']} is: {token_balance}")
        time.sleep(0.8)
        balances.append({
            'date': row['Date'],
            'block_number': block_number,
            'token_address': token_address,
            'token_name': row2['coingecko_id'],
            'token_decimals': row2['decimals'],
            'balance': token_balance / 10**token_decimals,
            'has_error': has_error
        })

L1ERC20Gateway_token_balances_per_day_part03 = pd.DataFrame(balances)


In [13]:
#L1ERC20Gateway_token_balances_per_day_part03.to_csv('L1ERC20Gateway/L1ERC20Gateway_token_balances_per_day_part03.csv', index=False)

<h1 style="color: blue; font-size: 36px; font-family: Arial;">Custom Gateway Escrow</h1>
Generic escrow

In [ ]:
#contract-address: 0xcEe284F754E854890e311e3280b767F80797180d
#created at Jun-16-2021 #at block 12647126


In [21]:
import pandas as pd
import requests

# Replace API_KEY with your own Ethplorer API key
API_KEY = "YOUR-API-KEY"

# Replace CONTRACT_ADDRESS with the address of the contract you want to query
CONTRACT_ADDRESS = "0xcEe284F754E854890e311e3280b767F80797180d"

# Construct the API endpoint URL
url = f"https://api.ethplorer.io/getAddressInfo/{CONTRACT_ADDRESS}?apiKey={API_KEY}"

# Send the API request and parse the response as JSON
response = requests.get(url)
data = response.json()

# Get the list of token balances held by the contract
token_balances = data.get("tokens", [])

# Create an empty DataFrame
token_addresses_held_L1CustomGateway = pd.DataFrame(columns=["Token Address"])

# Append each token address to a new row in the DataFrame
for token_balance in token_balances:
    token_address = token_balance.get("tokenInfo", {}).get("address")
    token_addresses_held_L1CustomGateway = token_addresses_held_L1CustomGateway.append({"Token Address": token_address}, ignore_index=True)

# Print the DataFrame
print(token_addresses_held_L1CustomGateway)


                                 Token Address
0   0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48
1   0x4e352cf164e64adcbad318c3a1e222e9eba4ce42
2   0x3642c0680329ae3e103e2b5ab29ddfed4d43cbe5
3   0xdac17f958d2ee523a2206206994597c13d831ec7
4   0x3258cd8134b6b28e814772dd91d5ecceea512818
5   0x431ad2ff6a9c365805ebad47ee021148d6f7dbe0
6   0x0ff5a8451a839f5f0bb3562689d9a44089738d11
7   0x469eda64aed3a3ad6f868c44564291aa415cb1d9
8   0xc4e15973e6ff2a35cc804c2cf9d2a1b817a8b40f
9   0xcc069f8323858537158dc47f6c208c8174038af9
10  0x3cd1833ce959e087d0ef0cb45ed06bffe60f23ba
11  0x2a95fe4c7e64e09856989f9ea0b57b9ab5f770cb
12  0xeabb8996ea1662cad2f7fb715127852cd3262ae9
13  0x58b6a8a3302369daec383334672404ee733ab239
14  0x20c375e822b6264e22941b74943f940a1cfe5f25
15  0x1fed524867348ecea0a4d9aeae989794dbf8f631
16  0x141c7c124e574a9be8a2ee654ac3b0ddc2d884be
17  0x7379cbce70bba5a9871f97d33b391afba377e885
18  0x17a10104cbc1ed155d083ead9fcf5c3440bb50e8
19  0x1f068a896560632a4d2e05044bd7f03834f1a465
20  0x57b9460

all token_addresses held by the escrow that also appear in the top 300 of CoinGecko

In [19]:
# all token_addresses held by the escrow that also appear in the top 300 of CoinGecko
L1CustomGateway_top_token_addresses = pd.merge(top_tokens_addresses, token_addresses_held_L1CustomGateway, left_on="token_address", right_on="Token Address")
L1CustomGateway_top_token_addresses['token_address'] = L1CustomGateway_top_token_addresses['token_address'].apply(lambda x: web3.to_checksum_address(x))


In [ ]:

def get_token_balance(token_holding_address_checksum, token_address, contract, block_number):
    try:
        token_balance = contract.functions.balanceOf(token_holding_address_checksum).call(block_identifier=block_number)
        has_error = False
    except Exception as e:
        token_balance = 0
        has_error = True
        print(f"Error getting token balance: {e}")
    return token_balance, has_error

# Define the ERC20 contract ABI
abi = [
    {
        'inputs': [],
        'name': 'decimals',
        'outputs': [{'internalType': 'uint8', 'name': '', 'type': 'uint8'}],
        'stateMutability': 'view',
        'type': 'function'
    },
    {
        'inputs': [{'internalType': 'address', 'name': '', 'type': 'address'}],
        'name': 'balanceOf',
        'outputs': [{'internalType': 'uint256', 'name': '', 'type': 'uint256'}],
        'stateMutability': 'view',
        'type': 'function'
    }
]

# Define the token addresses DataFrame
token_addresses_df = L1CustomGateway_top_token_addresses

token_holding_address = "0xcEe284F754E854890e311e3280b767F80797180d"
token_holding_address_checksum = web3.to_checksum_address(token_holding_address)

# Create a contract instance for each token address
contracts = {}
for _, row in token_addresses_df.iterrows():
    token_address = row['Token Address']
    token_address_checksum = web3.to_checksum_address(token_address)
    contract = web3.eth.contract(address=token_address_checksum, abi=abi)
    contracts[token_address] = contract

# Retrieve the token balances for each token address and block number, and store the results in a DataFrame
balances = []
for i, row in block_numbers_df.iloc[532:].iterrows():
    print(f"Retrieving token balances for {row['Date']} - Block Number:{row['Block Number']} ")
    for _, row2 in token_addresses_df.iterrows(): 
        token_decimals = row2['decimals']
        token_address = row2['Token Address']
        block_number = row['Block Number']
        contract = contracts[token_address]
        token_balance, has_error = get_token_balance(token_holding_address_checksum, token_address, contract, block_number)
        #print(f"On day {row['Date']} token address {token_address} had a balance of {token_balance} ")
        #print(f"Token balance for address {token_address} at Block-Nr. {row['Block Number']} is: {token_balance}")
        time.sleep(0.5)
        balances.append({
            'date': row['Date'],
            'block_number': block_number,
            'token_address': token_address,
            'token_name': row2['coingecko_id'],
            'token_decimals': row2['decimals'],
            'balance': token_balance / 10**token_decimals,
            'has_error': has_error
        })
        
L1CustomGateway_token_balances_per_day = pd.DataFrame(balances)


In [29]:
#L1CustomGateway_token_balances_per_day.to_csv('L1CustomGateway/L1CustomGateway_token_balances_per_day.csv', index=False)


<h1 style="color: blue; font-size: 36px; font-family: Arial;">Bridge Escrow (ETH)</h1>
ETH escrow
contract address: 

In [ ]:
#contract-address: 0x8315177aB297bA92A06054cE80a67Ed4DBd7ed3a
#created at Aug-25-2022 #at block 15411056


## Retreive ETH Balance of the escrow

In [ ]:
# New function to retrieve Ether balance of a smart contract at a specific block number
def get_contract_eth_balance(escrow_checksum, block_number):
    return web3.eth.get_balance(escrow_checksum, block_identifier=block_number)


escrow = "0x8315177aB297bA92A06054cE80a67Ed4DBd7ed3a"
escrow_checksum = web3.to_checksum_address(escrow)

# Retrieve the token and Ether balances for each token address and block number, and store the results in a DataFrame
token_address = "ether"
token_name = "ethereum"
token_decimals = 18

balances = []
for i, row in block_numbers_df.iloc[967:].iterrows():

    block_number = row['Block Number']
    eth_balance = get_contract_eth_balance(escrow_checksum, block_number) / 10**token_decimals # Convert Wei to Ether
    print(f"Retrieving balances for {row['Date']} - Block Number:{row['Block Number']} - balance:{eth_balance}  ")    
    balances.append({
            'date': row['Date'],
            'block_number': block_number,
            'token_address': token_address,
            'token_name': token_name,
            'token_decimals': token_decimals,
            'token_balance': eth_balance,
        })
    time.sleep(0.5)
        
L1Bridge_balances_per_day = pd.DataFrame(balances)


In [28]:
#L1Bridge_balances_per_day.to_csv('L1Bridge/L1Bridge_balances_per_day.csv', index=False)

<h1 style="color: blue; font-size: 36px; font-family: Arial;">DAI Escrow (DAI)</h1>
DAI escrow

In [ ]:
#contract-address: 0xA10c7CE4b876998858b1a9E12b10092229539400
#created at Sep-20-2021 #at block 13262081


## Retreive DAI Balance of the escrow

In [ ]:
def get_token_balance(token_holding_address_checksum, token_address, contract, block_number):
    try:
        token_balance = contract.functions.balanceOf(token_holding_address_checksum).call(block_identifier=block_number)
        has_error = False
    except Exception as e:
        token_balance = 0
        has_error = True
        print(f"Error getting token balance: {e}")
    return token_balance, has_error

# Define the ERC20 contract ABI
abi = [
    {
        'inputs': [],
        'name': 'decimals',
        'outputs': [{'internalType': 'uint8', 'name': '', 'type': 'uint8'}],
        'stateMutability': 'view',
        'type': 'function'
    },
    {
        'inputs': [{'internalType': 'address', 'name': '', 'type': 'address'}],
        'name': 'balanceOf',
        'outputs': [{'internalType': 'uint256', 'name': '', 'type': 'uint256'}],
        'stateMutability': 'view',
        'type': 'function'
    }
]

# Define the token address
token_address = "0x6B175474E89094C44Da98b954EedeAC495271d0F"
token_address_checksum = web3.to_checksum_address(token_address)

token_holding_address = "0xA10c7CE4b876998858b1a9E12b10092229539400"
token_holding_address_checksum = web3.to_checksum_address(token_holding_address)

# Create a contract instance for the token address
contract = web3.eth.contract(address=token_address_checksum, abi=abi)

# Retrieve the token balances for each block number, and store the results in a DataFrame
balances = []
for i, row in block_numbers_df.iloc[553:].iterrows():
    
    token_decimals = 18  # Assuming 18 decimal places for the DAI token
    block_number = row['Block Number']
    token_balance, has_error = get_token_balance(token_holding_address_checksum, token_address, contract, block_number)
    print(f"Retrieving token balance for {row['Date']} - Block Number:{row['Block Number']} - balance: {token_balance} ")
    time.sleep(0.5)
    balances.append({
        'date': row['Date'],
        'block_number': block_number,
        'token_address': token_address,
        'token_name': 'dai',  
        'token_decimals': token_decimals,
        'balance': token_balance / 10**token_decimals,
        'has_error': has_error
    })

L1DaiEscrow_balances_per_day = pd.DataFrame(balances)


In [ ]:
L1DaiEscrow_balances_per_day

In [33]:
#L1DaiEscrow_balances_per_day.to_csv('L1DaiEscrow/L1DaiEscrow_balances_per_day.csv', index=False)


<h1 style="color: blue; font-size: 36px; font-family: Arial;">OLD Bridge (ETH)</h1>
for historic data

In [ ]:
#contract-address: 0x011B6E24FfB0B5f5fCc564cf4183C5BBBc96D515
#created at May-28-2021 at block 12525700


In [ ]:

# New function to retrieve Ether balance of a smart contract at a specific block number
def get_contract_eth_balance(escrow_checksum, block_number):
    return web3.eth.get_balance(escrow_checksum, block_identifier=block_number)


escrow = "0x011B6E24FfB0B5f5fCc564cf4183C5BBBc96D515"
escrow_checksum = web3.to_checksum_address(escrow)

# Retrieve the token and Ether balances for each token address and block number, and store the results in a DataFrame
token_address = "ether"
token_name = "ethereum"
token_decimals = 18

#May-28-2021
balances = []
for i, row in block_numbers_df.iloc[513:].iterrows():

    block_number = row['Block Number']
    eth_balance = get_contract_eth_balance(escrow_checksum, block_number) / 10**token_decimals # Convert Wei to Ether
    print(f"Retrieving balances for {row['Date']} - Block Number:{row['Block Number']} - balance:{eth_balance}  ")    
    balances.append({
            'date': row['Date'],
            'block_number': block_number,
            'token_address': token_address,
            'token_name': token_name,
            'token_decimals': token_decimals,
            'token_balance': eth_balance,
        })
    time.sleep(0.5)
        
L1OldBridge_balances_per_day = pd.DataFrame(balances)


In [99]:
#L1OldBridge_balances_per_day.to_csv('L1OldBridge/L1OldBridge_balances_per_day.csv', index=False)

<h1 style="color: blue; font-size: 36px; font-family: Arial;">Combining the Escrows</h1>


In [12]:
L1ERC20Gateway_token_balances_per_day_part01 = pd.read_csv('L1ERC20Gateway/L1ERC20Gateway_token_balances_per_day_part01.csv') 
L1ERC20Gateway_token_balances_per_day_part02 = pd.read_csv('L1ERC20Gateway/L1ERC20Gateway_token_balances_per_day_part02.csv') 
L1ERC20Gateway_token_balances_per_day_part03 = pd.read_csv('L1ERC20Gateway/L1ERC20Gateway_token_balances_per_day_part03.csv') 


L1ERC20Gateway_token_balances_per_day_part01['escrow_address'] = '0xa3A7B6F88361F48403514059F1F16C8E78d60EeC'
L1ERC20Gateway_token_balances_per_day_part01['escrow_address_rdbl'] = 'L1ERC20Gateway'
L1ERC20Gateway_token_balances_per_day_part01 = L1ERC20Gateway_token_balances_per_day_part01.drop(columns=['has_error'])

L1ERC20Gateway_token_balances_per_day_part02['escrow_address'] = '0xa3A7B6F88361F48403514059F1F16C8E78d60EeC'
L1ERC20Gateway_token_balances_per_day_part02['escrow_address_rdbl'] = 'L1ERC20Gateway'
L1ERC20Gateway_token_balances_per_day_part02 = L1ERC20Gateway_token_balances_per_day_part02.drop(columns=['has_error'])


L1ERC20Gateway_token_balances_per_day_part03['escrow_address'] = '0xa3A7B6F88361F48403514059F1F16C8E78d60EeC'
L1ERC20Gateway_token_balances_per_day_part03['escrow_address_rdbl'] = 'L1ERC20Gateway'
L1ERC20Gateway_token_balances_per_day_part03 = L1ERC20Gateway_token_balances_per_day_part03.drop(columns=['has_error'])


In [13]:
L1CustomGateway_token_balances_per_day = pd.read_csv('L1CustomGateway/L1CustomGateway_token_balances_per_day.csv')

L1CustomGateway_token_balances_per_day['escrow_address'] = '0xcEe284F754E854890e311e3280b767F80797180d'
L1CustomGateway_token_balances_per_day['escrow_address_rdbl'] = 'L1CustomGateway'
L1CustomGateway_token_balances_per_day = L1CustomGateway_token_balances_per_day.drop(columns=['has_error'])



In [14]:
L1Bridge_balances_per_day = pd.read_csv('L1Bridge/L1Bridge_balances_per_day.csv')

L1Bridge_balances_per_day = L1Bridge_balances_per_day.rename(columns={'token_balance': 'balance'})
L1Bridge_balances_per_day['escrow_address'] = '0x8315177aB297bA92A06054cE80a67Ed4DBd7ed3a'
L1Bridge_balances_per_day['escrow_address_rdbl'] = 'L1Bridge'

In [15]:
L1OldBridge_balances_per_day = pd.read_csv('L1OldBridge/L1OldBridge_balances_per_day.csv')

L1OldBridge_balances_per_day = L1OldBridge_balances_per_day.rename(columns={'token_balance': 'balance'})
L1OldBridge_balances_per_day['escrow_address'] = '0x011B6E24FfB0B5f5fCc564cf4183C5BBBc96D515'
L1OldBridge_balances_per_day['escrow_address_rdbl'] = 'L1OldBridge'

In [16]:
L1DaiEscrow_balances_per_day = pd.read_csv('L1DaiEscrow/L1DaiEscrow_balances_per_day.csv')

L1DaiEscrow_balances_per_day['escrow_address'] = '0xA10c7CE4b876998858b1a9E12b10092229539400'
L1DaiEscrow_balances_per_day['escrow_address_rdbl'] = 'L1DaiGateway'
L1DaiEscrow_balances_per_day = L1DaiEscrow_balances_per_day.drop(columns=['has_error'])


In [17]:
# create a list of the dataframes
dfs = [L1ERC20Gateway_token_balances_per_day_part01,
       L1ERC20Gateway_token_balances_per_day_part02,
       L1ERC20Gateway_token_balances_per_day_part03,
       L1CustomGateway_token_balances_per_day, 
       L1Bridge_balances_per_day,
       L1OldBridge_balances_per_day,
       L1DaiEscrow_balances_per_day]

# concatenate the dataframes along the rows
arbitrum_escrow_balances = pd.concat(dfs, ignore_index=True)

In [20]:
arbitrum_escrow_balances = arbitrum_escrow_balances[arbitrum_escrow_balances['date'] < '2023-04-30']
arbitrum_escrow_balances['token_address'] = arbitrum_escrow_balances['token_address'].apply(lambda x: web3.to_checksum_address(x) if x != 'ether' else x)

price_data = pd.read_csv('../../utils/price-data/price_data.csv') 
price_data['token_address'] = price_data['token_address'].apply(lambda x: web3.to_checksum_address(x) if x != 'ether' else x)


In [21]:
arbitrum_escrow_balances['date'] = pd.to_datetime(arbitrum_escrow_balances['date'])
price_data['timestamp'] = pd.to_datetime(price_data['timestamp'])
price_data['token_address'] = price_data['token_address'].astype(str)
arbitrum_escrow_balances['token_address'] = arbitrum_escrow_balances['token_address'].astype(str)

#### there are some rows that could not have merged, due to missing price data on CoinGecko. 
In some instances, CoinGecko did not immediately list certain tokens following the deployment of their smart contracts resulting in a small number of days that could not be taken into account for our analysis

In [22]:
merged_df = arbitrum_escrow_balances.merge(price_data, left_on=['token_address', 'date'], right_on=['token_address', 'timestamp'], how='left')
missing_values = merged_df[['timestamp', 'price']].isna().any()
print(missing_values)

timestamp    True
price        True
dtype: bool


In [23]:
arbitrum_escrows_final = arbitrum_escrow_balances.merge(price_data, left_on=['token_address', 'date'], right_on=['token_address', 'timestamp'])

In [25]:
arbitrum_escrows_final['balance_usd'] = arbitrum_escrows_final['balance'] * arbitrum_escrows_final['price']

In [5]:
#can be disregarded and will be overwritten with the correct value in eth in the analysis part
arbitrum_escrows_final['token_amount'] = arbitrum_escrows_final['balance'] * 10 ** arbitrum_escrows_final['token_decimals']

def wei_to_eth(wei_value):
    # 1 Ether (ETH) is equivalent to 10^18 Wei
    eth_value = wei_value * 1e-18
    return eth_value

# Apply the conversion function and add a new column
arbitrum_escrows_final["balance_eth"] = arbitrum_escrows_final["token_amount"].apply(wei_to_eth)

### save final df

In [24]:
#arbitrum_escrows_final.to_csv('../../data/arbitrum_escrows_final.csv', index=False)